## **Codificación de máscaras de coco**
El siguiente código recorre todas las carpetas de escenas en la carpeta de entrada y crea un marco de datos pd con el ID, Ancho, Alto, EncodedMasks de cada escena. \
Las máscaras codificadas de la misma escena se dividen con un espacio. \
La carpeta que desea codificar debe especificarse a continuación en la variable "name_of_the_dataset".

In [1]:
# Este entorno Python 3 viene con muchas bibliotecas analíticas útiles instaladas
# Está definido por la imagen Docker kaggle/python:https://github.com/kaggle/docker-python
# Por ejemplo, aquí hay varios paquetes útiles para cargar

import numpy as np # librería para hacer calculos de álgebra lineal
import pandas as pd # tratamiento de datos, archivo CSV  I/O (ej. pd.read_csv)
import os
import zlib
import base64
import typing as t
import cv2
import matplotlib.pyplot as plt
!pip install pycocotools
from pycocotools import _mask as coco_mask

# Los archivos de datos de entrada están disponibles en la sección de sólo lectura "../input/" directory
# Por ejemplo, ejecutando esto (haciendo clic en ejecutar o pulsando  Shift+Enter) se listarán todos los archivos del directorio de entrada

# Puedes escribir hasta 20 GB en el directorio actual(/kaggle/working/) que se conserva como salida cuando se crea una versión utilizando "Save & Run All" 
# También puedes escribir archivos temporales en /kaggle/temp/, pero no se guardarán fuera de la sesión actual

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 2.3 MB/s eta 0:00:00


In [2]:
# Ruta al directorio de entrada
name_of_the_dataset = "VALIDATION" # elige entre 'TRAIN' O 'VALIDATION'
input_directory = '/kaggle/input/5r-competicion/' + name_of_the_dataset + "/" + name_of_the_dataset


In [3]:
# Ruta al directorio de entrada
def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Convierte una máscara binaria a texto ascii de codificación de desafío OID."""
    # Comprobar máscara de entrada
    if mask.dtype != np.uint8:
        raise ValueError(
            "encode_binary_mask espera una máscara binaria, dtype recibido == %s" % mask.dtype
        )

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask espera una máscara 2d, forma recibida == %s" % mask.shape
        )

    # Convertir la máscara de entrada en la entrada prevista de COCO API 
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # codoficar máscara RLE 
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # Compresión y codificación base64
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str.decode("utf-8")  # Devolver como cadena

# Inicializar un diccionario para almacenar las máscaras codificadas
encoded_masks_dict = {}

# Recorrer todas las carpetas del directorio de entrada
for root, dirs, files in os.walk(input_directory):
    try:
        dirs.remove('all')
        print(f"Carpeta 'all'eliminada en {root}")
    except:
        print(f"Carpeta 'all' no encontrada en {root}")
    for dir_name in dirs:
        print(f"Procesando carpeta {dir_name} ...")
        folder_path = os.path.join(root, dir_name)
        # Inicializar las listas para almacenar las máscaras de cada carpeta
        encoded_masks_str = ""
        binary_masks_list = []
        first_image_path = None
        width = None
        height = None
        # Recorrer todos los archivos de la carpeta actual
        for filename in os.listdir(folder_path):
            if ((filename.lower().endswith('.png')) and ("Obj" in filename)):
                # Leer la imagen
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Lectura en escala de grises

                # Obtener anchura y altura de la imagen
                height, width = image.shape

                # Convertir a máscara binaria  (thresholding)
                _, binary_mask = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)

                # Encode the binary mask
                encoded_mask = encode_binary_mask(binary_mask)

                # Codificar la máscara binaria
                if encoded_masks_str:
                    encoded_masks_str += " " + encoded_mask
                else:
                    encoded_masks_str = encoded_mask

                # Añade la máscara codificada y la máscara binaria a las listas
                binary_masks_list.append(binary_mask)
        print(f"La codificación de la carpeta {dir_name} se ha realizado correctamente!")
        # Añade el nombre de la carpeta como ID de la imagen y su información asociada al diccionario
        encoded_masks_dict[dir_name] = {
            'Width': width,    # Ancho
            'Height': height,  # Alto
            'EncodedMasks': encoded_masks_str,   # máscaras codificadas
            'Masks': binary_masks_list  # lista de máscaras binarias
        }
        print("Codificación finalizada!")

Carpeta 'all'eliminada en /kaggle/input/5r-competicion/VALIDATION/VALIDATION
Procesando carpeta 20240321_152445 ...
La codificación de la carpeta 20240321_152445 se ha realizado correctamente!
Codificación finalizada!
Procesando carpeta 20240321_152451 ...
La codificación de la carpeta 20240321_152451 se ha realizado correctamente!
Codificación finalizada!
Procesando carpeta 20240321_152508 ...
La codificación de la carpeta 20240321_152508 se ha realizado correctamente!
Codificación finalizada!
Procesando carpeta 20240321_152456 ...
La codificación de la carpeta 20240321_152456 se ha realizado correctamente!
Codificación finalizada!
Procesando carpeta 20240321_152656 ...
La codificación de la carpeta 20240321_152656 se ha realizado correctamente!
Codificación finalizada!
Procesando carpeta 20240321_152712 ...
La codificación de la carpeta 20240321_152712 se ha realizado correctamente!
Codificación finalizada!
Procesando carpeta 20240321_152502 ...
La codificación de la carpeta 20240321

In [4]:
# Crear un DataFrame para exportación CSV (excluyendo el 'DecodedMask', máscaras decodificadas)
df = pd.DataFrame.from_dict(encoded_masks_dict, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'ID'}, inplace=True)

# Eliminar la columna "DecodedMask" del DataFrame
df.drop(columns=['Masks'], inplace=True)

# Guardar el DataFrame en un archivo CSV
df.to_csv("./" + name_of_the_dataset + "_COCO_COMPRESSED.csv", index=False)

# Visualizar el DataFrame
df.head()


,ID,Width,Height,EncodedMasks
0,20240321_152445,2208,1242,eNo9T9EOgjAM/KXeQPDFGH0wAbbOmEk0qIBBQliC///mNs...
1,20240321_152451,2208,1242,eNplkVtTwkAMhf/SBgrOKNNBeXEsTeykF0FoQbmb1v//Zg...
2,20240321_152508,2208,1242,eNpVT8sOwjAM+6U6LRIcOCOxLZlgIB6DTRsw8fj/M0k7BK...
3,20240321_152456,2208,1242,eNpNUGtTwyAQ/EvskdrO+N3xwR0N0GhiE2uiZszD/P9vQu...
4,20240321_152656,2208,1242,eNpFzksOgkAQhOErdQ1s0AOgBqp1hkDwGR8YZAbvv7SjCx...
